In [91]:
from histdata import download_hist_data as dl
from histdata.api import Platform as P, TimeFrame as TF
import pandas as pd
import numpy as np
from datetime import datetime as dt
import zipfile
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import gc
import os, shutil

# Functions

In [58]:
def dl_data(year, pair, platform=P.GENERIC_ASCII, time_frame=TF.ONE_MINUTE):
    year = str(year)
    zip_name = dl(year=year, month=None, pair=pair, platform=platform, time_frame=time_frame)[2:]
    with zipfile.ZipFile(zip_name,"r") as zip_ref:
        zip_ref.extractall("temp")
    csv_name = zip_name.replace('zip','csv')
    df = pd.read_csv(f'temp/{csv_name}',header=None,sep=';')
    return df

In [84]:
def connect_db(db_url):
    engine = create_engine(db_url, echo=False)
    if not database_exists(engine.url):
        create_database(engine.url)
    return engine

In [85]:
def clean_data(data):
    columns = ['Datetime','Open','High','Low','Close','Volume']
    data.columns = columns
    data['Datetime'] = pd.to_datetime(data['Datetime'],format='%Y%m%d %H%M%S')
    data.sort_index(inplace=True)
    return data

In [89]:
def clear_temp(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Main

In [92]:
pairs = ['audusd','eurusd','gbpusd','nzdusd','usdcad','usdchf','usdjpy']

db_url = 'sqlite:///fx_min_data.db'
db_engine = connect_db(db_url)

for pair in pairs:
    # download data by year
    data_list = list()
    for year in range(2000,2024):
        data_list.append(dl_data(year,pair))

    # clean data and save to sql
    data = clean_data(pd.concat(data_list,axis=0))
    data.to_sql(pair, con=db_engine)
    
    # free memory
    del data_list
    del data
    gc.collect()

    # delete temp files
    clear_temp('temp')

https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2000
{'tk': '55a31ada7f44194623c4f93940a19622', 'date': '2000', 'datemonth': '2000', 'platform': 'ASCII', 'timeframe': 'M1', 'fxpair': 'AUDUSD'}
Wrote to .\DAT_ASCII_AUDUSD_M1_2000.zip
https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2001
{'tk': '5ed57d02a7f1aee7dde2d506ef03b1e3', 'date': '2001', 'datemonth': '2001', 'platform': 'ASCII', 'timeframe': 'M1', 'fxpair': 'AUDUSD'}
Wrote to .\DAT_ASCII_AUDUSD_M1_2001.zip
https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2002
{'tk': '6eb7809341fdf77e200f963564488676', 'date': '2002', 'datemonth': '2002', 'platform': 'ASCII', 'timeframe': 'M1', 'fxpair': 'AUDUSD'}
Wrote to .\DAT_ASCII_AUDUSD_M1_2002.zip
https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2003
{'tk': '4ac032c232bd91ffe33130cc7fff5ec5', 'date': '2003

AssertionError: There is no token. Please make sure your year/month/pair is correct.Example is year=2016, month=7, pair=eurgbp

In [83]:
sample_sql_database = engine.execute(f"SELECT * FROM {pair} LIMIT 10").fetchall()

print(sample_sql_database)

[(0, '2022-01-02 17:03:00.000000', 1.1369, 1.1369, 1.1369, 1.1369, 0), (0, '2023-01-01 17:04:00.000000', 1.0697, 1.06974, 1.0697, 1.0697, 0), (1, '2022-01-02 17:04:00.000000', 1.13689, 1.13689, 1.13689, 1.13689, 0), (1, '2023-01-01 17:05:00.000000', 1.06973, 1.06978, 1.0697, 1.06971, 0), (2, '2022-01-02 17:05:00.000000', 1.13692, 1.13692, 1.13692, 1.13692, 0), (2, '2023-01-01 17:06:00.000000', 1.06966, 1.06966, 1.06966, 1.06966, 0), (3, '2022-01-02 17:06:00.000000', 1.13668, 1.13671, 1.13652, 1.13652, 0), (3, '2023-01-01 17:08:00.000000', 1.0697, 1.06974, 1.0697, 1.06974, 0), (4, '2022-01-02 17:07:00.000000', 1.13655, 1.13692, 1.13649, 1.13682, 0), (4, '2023-01-01 17:10:00.000000', 1.06975, 1.0698, 1.06972, 1.06972, 0)]
